In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('LoanStats3d.csv', skipinitialspace=True, header=1)

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421092,36271333,38982739.0,13000.0,13000.0,13000.0,60 months,15.99%,316.07,D,D2,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806,39222577.0,12000.0,12000.0,12000.0,60 months,19.99%,317.86,E,E3,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262,38982659.0,20000.0,20000.0,20000.0,36 months,11.99%,664.20,B,B5,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I already analyzed and chose what I believe are the best features. I did this in the previous assignment. These features are 'last_pymnt_amnt' and 'out_prncp'.

In [4]:
features = ['last_pymnt_amnt', 'out_prncp']
features = df[features]

In [5]:
# Let's check the percentage missing values with our features and target variable
df[['last_pymnt_amnt', 'out_prncp', 'loan_status']].isnull().sum()/df[['last_pymnt_amnt', 'out_prncp', 'loan_status']].isnull().count()

last_pymnt_amnt    0.000005
out_prncp          0.000005
loan_status        0.000005
dtype: float64

In [6]:
# Because there is a insignificant percentage of missing values in our features, 
# I will drop the rows with missing values

df.dropna(subset=['last_pymnt_amnt', 'out_prncp', 'loan_status'], inplace=True)

In [20]:
import time
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

X = df[['out_prncp', 'last_pymnt_amnt']]
Y = df['loan_status']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=200)

start_time = time.time()
decision_tree = tree.DecisionTreeClassifier(
    criterion='gini', # ASk Mukesh --- How to choose entropy or gini impurity???
    max_features=1,
    max_depth=5
)

decision_tree.fit(X_train, Y_train)
print(decision_tree.score(X_train, Y_train))
print(decision_tree.score(X_test, Y_test))
print("--- %s seconds ---" % (time.time() - start_time))

0.9466094349256106
0.9482183355299872
--- 1.5452139377593994 seconds ---


In [19]:
from sklearn import ensemble

start_time = time.time()

rfc = ensemble.RandomForestClassifier(
    n_estimators=10,
    criterion='gini',
    max_features=1,
    max_depth=5
)
rfc.fit(X_train, Y_train)
print(rfc.score(X_train, Y_train))
print(rfc.score(X_test, Y_test))
print("--- %s seconds ---" % (time.time() - start_time))

0.946621308730809
0.948277704555979
--- 4.1596598625183105 seconds ---


It appears that the decision tree performs just as good as the random forest. The random forest is also about 3 times slower.